In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from llamabot import SimpleBot
from llamabot.prompt_library.diffbot import diffbot, get_github_diff

In [ ]:
url = "https://github.com/pyjanitor-devs/pyjanitor/pull/1262"

In [ ]:
from llamabot.prompt_library.diffbot import (
    suggest_improvements,
    diffbot,
)


diff = get_github_diff("https://github.com/pyjanitor-devs/pyjanitor/pull/1262")
print(diff)

We will use StructuredBot to generate a structured commit message based on the diff.

In [ ]:
from pydantic import BaseModel, Field, model_validator
from enum import Enum


class CommitType(str, Enum):
    fix = "fix"
    feat = "feat"
    build = "build"
    chore = "chore"
    ci = "ci"
    docs = "docs"
    style = "style"
    refactor = "refactor"
    perf = "perf"
    test = "test"
    other = "other"


class DescriptionEntry(BaseModel):
    txt: str = Field(
        ...,
        description="A single bullet point describing one major change in the commit.",
    )

    @model_validator(mode="after")
    def validate_description(self, value):
        if len(self.txt) > 79:
            raise ValueError(
                "Description should be less than or equal to 79 characters."
            )
        return self


class CommitMessage(BaseModel):
    commit_type: CommitType = Field(
        ...,
        description="Type of change. Should usually be fix or feat. But others, based on the Angular convention, are allowed, such as build, chore, ci, docs, style, refactor, perf, test, and others.",
    )
    scope: str = Field(
        ...,
        description="Scope of change. If commits are only in a single file, then scope should be the filename. If commits involve multiple files, then the scope should be one word that accurately describes the scope of changes.",
    )
    description: str = Field(
        ...,
        description="A one line description of the changes, in <79 characters.",
    )

    body: list[DescriptionEntry] = Field(
        ...,
        description="A list of description entries. Each description entry should have a single bullet point describing one major change in the commit. At most 6 entries.",
    )

    breaking_change: bool = Field(
        ..., description="Whether or not there is a breaking change in the commit. "
    )

    footer: str = Field("", description="An optional footer.")

    @model_validator(mode="after")
    def validate_scope(self):
        if len(self.scope) > 0 and len(self.scope.split()) > 1:
            raise ValueError("Scope should be one word.")
        return self

    @model_validator(mode="after")
    def validate_body(self):
        if len(self.body) > 6:
            raise ValueError("Description entries should be no more than 6.")
        return self

    def format(self) -> str:
        return _fmt(self)


@prompt
def _fmt(cm):
    """{{ cm.commit_type.value }}({{ cm.scope }}){%if cm.breaking_change %}!{% else %}{% endif %}: {{ cm.description }}

    {% for bullet in cm.body %}- {{ bullet.txt }}
    {% endfor %}

    {% if cm.footer %}{{ cm.footer }}{% endif %}
    """

In [ ]:
from llamabot import StructuredBot

bot = StructuredBot(
    system_prompt="You are an expert software developer who writes excellent and accurate commit messages. You are going to be given a diff as input, and you will generate a structured JSON output based on the pydantic model provided.",
    pydantic_model=CommitMessage,
    model_name="groq/llama-3.1-70b-versatile",
    stream_target="none",
)

result = bot(diff)

In [ ]:
print(result.format())

Old stuff below.

In [ ]:
@prompt
def structured_commit_message(diff):
    """Please write a commit message for the following diff.

    {{ diff }}

    # noqa: DAR101

    Use the Conventional Commits specification to write the diff.

    Return it for me in JSON format:

    {
        "type": "<type>",
        "scope": "<scope>",
        "description": "<description>",
        "breaking_change": "<breaking_change>",
        "body": ["<bullet point 1 no bullet>", "<bullet point 2 no bullet>",...],
        "footer": "<footer>"
    }

    The commit contains the following structural elements,
    to communicate intent to the consumers of your library:

    fix: a commit of the type fix patches a bug in your codebase
        (this correlates with PATCH in Semantic Versioning).
    feat: a commit of the type feat introduces a new feature to the codebase
        (this correlates with MINOR in Semantic Versioning).
    BREAKING CHANGE: a commit that has a footer BREAKING CHANGE:,
        or appends a ! after the type/scope,
        introduces a breaking API change
        (correlating with MAJOR in Semantic Versioning).
        A BREAKING CHANGE can be part of commits of any type.

    types other than fix: and feat: are allowed,
    for example @commitlint/config-conventional
    (based on the Angular convention) recommends
    build:, chore:, ci:, docs:, style:, refactor:, perf:, test:, and others.

    footers other than BREAKING CHANGE: <description> may be provided
    and follow a convention similar to git trailer format.

    Additional types are not mandated by the Conventional Commits specification,
    and have no implicit effect in Semantic Versioning
    (unless they include a BREAKING CHANGE).
    A scope may be provided to a commit's type,
    to provide additional contextual information and is contained within parenthesis,
    e.g., feat(parser): add ability to parse arrays.
    Within the optional body section, prefer the use of bullet points.

    Final instructions:

    1. Do not fence the commit message with back-ticks or quotation marks.
    2. Do not add any other text except the commit message itself.
    3. Only write out the commit message.
    """

In [ ]:
from llamabot import SimpleBot


bot = SimpleBot(
    "You are an expert Git user",
    json_mode=True,
)
response = bot(structured_commit_message(diff))

In [ ]:
import json


@prompt
def rehydrate_commit_message(commit_json):
    """{{ commit_json["type"] }}({{ commit_json["scope"] }}): {{ commit_json["description"] }}
    {% for item in commit_json["body"] %}
    - {{ item }}{% endfor %}

        {% if commit_json["footer"] %}
        {{ commit_json["footer"] }}
        {% endif %}
    """


print(rehydrate_commit_message(json.loads(response.content)))

In [ ]:
from llamabot import SimpleBot


bot = SimpleBot("You are an expert Git user", json_mode=True)
response = bot(structured_commit_message(diff))

In [ ]:
# print(write_commit_message(diff))

In [ ]:
bot = commitbot()
bot(write_commit_message(diff))
# diffbot(describe_advantages(diff))

In [ ]:
diffbot(suggest_improvements(diff))

In [ ]:
asdfasdfadsf